# Sistema de recomendación: 
## www.ElResumen.com

En este decumeno se va a crear un sistema de recomendación de los libros del portal *elresumen.com* haciendo uso del **Procesamiento del Lenguaje Natural** a través de los resúmenes disponibles.

<img src="images/sr-books.jpg" width="600px" >

Las librerías utilizadas:

In [1]:
import nltk
import csv
from time import time
import numpy as np
from gensim import corpora, models, similarities
import gensim
import requests
from bs4 import BeautifulSoup
import threading

C:\Users\Rodrigo\Anaconda3\lib\site-packages\gensim\utils.py:855: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


Para la obtención de estos resumenes vamos a utilizar una técnica conocida como * Web Scraping* que consiste en, navegando a través del código HTML de una página web, extraer cierta información.

Para realizar ello lo primero tenemos que hacer es 'escrapear' en la página principal (http://www.elresumen.com/listado_de_libros.htm) todos los enlaces donde se encuentra el resumen de cada libro para posteriormente entrar en dicho enlace y así poder extraer su resumen.

Comenzamos obteniendo los enlaces.

In [2]:
url = "http://www.elresumen.com/listado_de_libros.htm"
soup = BeautifulSoup(requests.get(url).text)
nombre = []
lista = soup('tr')
link=[]

for i in lista:
    try:
        link.append(i.find('a').get('href'))
    except:
        pass

nones = link.count(None)
none = [link.remove(None) for k in range(nones)]

C:\Users\Rodrigo\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Users\Rodrigo\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


Para sacar información del HTML tenemos que traernos todo el código de la página a la memoria. Esto conlleva un cierto tiempo debido a la gran cantidad de enlaces que tenemos que "visitar", por lo que vamos a utilizar la función "Threads" de la librería *threading* para paralelizar.

Para comenzar tenemos que dividir el problema, en este caso como tenemos una lista con todos los enlaces, vamos a crear varios diccionarios donde en cada uno se encontrará una parte de los enlaces.

In [3]:
urls = ["http://www.elresumen.com/{}".format(l) for l in link]

u = len(urls)
n_paralelo = u//4

threads = list()
resu=[]
d = dict()
for b in range(n_paralelo):
    d['urls%1d' % (b+1)] = urls[b*u//(n_paralelo):u*(b+1)//(n_paralelo)]

In [4]:
t00 = time()
def worker(ur):
    w = 0
    for h in d[str(ur)]:
        soup2 = BeautifulSoup(requests.get(str(h)).text)
        re = soup2('main', 'main-content')
        resumen = []
        [nombre.append(a.find_all('h1')[1].text) for a in re]
        [[resumen.append(b.text) for b in a.find_all('p') if len(str(b)) > 100]
         for a in re]
    
        resumen.remove(resumen[len(resumen)-1])
        resumen.remove(resumen[len(resumen)-1])
        resumen.remove(resumen[len(resumen)-1])
        resumen.remove(resumen[len(resumen)-1])
        resumen.remove(resumen[0])
        
        n = len(resumen)
        
        resumen_bueno=[""]
        for p in range(n):
            resumen_bueno[0]+=resumen[p]
            
        resu.append(resumen_bueno)
        w+=1
        d1=(w/(len(d[str(ur)])))*100
        if ur == 'urls1':
            print("Completado: {} por ciento".format(round(d1,2)))
        else:
            pass
    return resu, nombre

for i in d:
    t = threading.Thread(target=worker, args=(i,))
    threads.append(t)
    t.start()
    
dic = dict()
for j in threads:
    j.join()
    for di in range(len(nombre)):
        dic[nombre[di]]=resu[di]

print("| Tiempo de scrapeo total: %0.4f s" % (time() - t00),"|")

C:\Users\Rodrigo\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 882 of the file C:\Users\Rodrigo\Anaconda3\lib\threading.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


Completado: 25.0 por ciento
Completado: 50.0 por ciento
Completado: 75.0 por ciento
Completado: 100.0 por ciento
| Tiempo de scrapeo total: 9.0107 s |


Podemos guardar los resumenes en un archivo csv. Aunque es opcional, es recomendable debido a que así ya no tendremos que escrapear los resumenes nuevamente.

In [5]:
resu_lib = open('ResumenesLibros.csv', 'w', newline='', encoding='utf-8')
salida = csv.writer(resu_lib)
for i in dic:
    salida.writerow([i, dic[i]])
del salida
resu_lib.close()

Ahora comenzaremos a limpiar los resumenes, ya que al bajarnos el código HTML suele venir con algunos símbolos. También seleccionaremos el tipo de separación que vamos a hacer (tokenizar, por palabras) y el idioma de los "StopWords". Los stop words son palabras que se suelen repetir mucho y que no nos dice nada del libro, como las conjunciones, artículos etc.

In [6]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')

from nltk.corpus import stopwords
stopWords = set(stopwords.words('spanish'))

from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer("spanish")

Importamos los resumenes del fichero generado.

In [7]:
def leerLibros(n_max):
    data = open('ResumenesLibros.csv', 'r', encoding='utf-8')
    resu_lib = csv.reader(data, delimiter=',')
    books = {}
    n = 0
    for row in resu_lib:
        books[str(row[0])]=row[1]
        if n > n_max:
            break
        n += 1
    data.close()
    return books

A continuación, crearemos las funciones que nos hará el preprocesado y limpieza de los resumenes de los libros. Para ello, recorremos todas las oraciones de un texto, que son los resumenes de los libros.

* nltk.word_tokenize devuelve la lista de palabras que forman la frase (tokenización).

* nltk.pos_tag devuelve el part of speech (categoría) correspondiente a la palabra introducida.

* nltk.ne_chunk devuelve la etiqueta correspondiente al part of speech

In [8]:
nombres = []
def obtenerNombresPropios(nombres, texto):

    for frase in nltk.sent_tokenize(texto):
        for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(frase))):
            try:
                if chunk.label() == 'PERSON':
                    for c in chunk.leaves():
                        if str(c[0].lower()) not in nombres:
                            nombres.append(str(c[0]).lower())
            except AttributeError:
                pass
    return nombres

Ahora eliminaremos los signos de puntuación usando tokenizer

In [9]:
def preprocesarLibros(libros):
    print("Preprocesando libros")
    nombresPropios = []

    for elemento in libros:

        libro = elemento

        resumen = libros[elemento]
        texto = ' '.join(tokenizer.tokenize(resumen))

        nombresPropios = obtenerNombresPropios(nombresPropios, texto)

    ignoraPalabras = stopWords
    ignoraPalabras.union(nombresPropios)

    palabras = [[]]
    for elemento in libros:
        libro = elemento
        textoPreprocesado = []
        for palabra in tokenizer.tokenize(texto):
            if (palabra.lower() not in ignoraPalabras):
                textoPreprocesado.append(stemmer.stem(palabra.lower()))
                palabras.append([(stemmer.stem(palabra.lower()))])

    return palabras

Una vez preprocesado el texto de los resuemesn podemos crear la colección de textos

In [10]:
def crearColeccionTextos(libros):
    print("Creando colección global de resúmenes")
    textos = []
    
    for elemento in libros:
        libro = elemento
        texto = libros[elemento]
        lista = texto.split(' ')

        textos.append(lista)

    return textos

Ahora crearemos el diccionario de palabras, que estará formado por la concatenación de todas las palabras que aparecen en el resumen de algun  libro.

In [11]:
def crearDiccionario(textos):
    print("Creación del diccionario global")
    return corpora.Dictionary(textos)

Ahora crearemos el corpus de los resúmenes preprocesados

In [12]:
def crearCorpus(diccionario):
    print("Creación del corpus global con los resúmenes de todos los libros")
    return [diccionario.doc2bow(texto) for texto in textos]

A continuación vamos a seleccionar el número de libros que queramos tratar

In [13]:
libros = leerLibros(50)
palabras = preprocesarLibros(libros)
textos = crearColeccionTextos(libros)
diccionario = crearDiccionario(textos)
corpus = crearCorpus(diccionario)

Preprocesando libros
Creando colección global de resúmenes
Creación del diccionario global
Creación del corpus global con los resúmenes de todos los libros


Creación del modelo TF-IDF (https://es.wikipedia.org/wiki/Tf-idf)

In [14]:
def crearTfIdf(corpus):
    print("Creación del Modelo Espacio-Vector Tf-Idf")
    tfidf = models.TfidfModel(corpus)
    corpus_tfidf = tfidf[corpus]
    return corpus_tfidf

Creación del modelo LSA (Latent Semantic Analysis). Añadiremos unos parámetros de control del recomendador en si.

In [15]:
TOTAL_TOPICOS_LSA = 30
UMBRAL_SIMILITUD = 0.85

In [16]:
def crearLSA(corpus,pel_tfidf):
    print("Creación del modelo LSA: Latent Semantic Analysis")
    numpy_matrix = gensim.matutils.corpus2dense(corpus,
                                                num_terms = 50000)
    svd = np.linalg.svd(numpy_matrix, full_matrices=False,
                        compute_uv = False)

    lsi = models.LsiModel(pel_tfidf, id2word=diccionario,
                          num_topics=TOTAL_TOPICOS_LSA)

    indice = similarities.MatrixSimilarity(lsi[pel_tfidf])

    return lsi,indice

In [17]:
def crearNombresLibros(libros):
    nombresLibros = []
    for elemento in libros:
        nombresLibros.append(elemento)
    return nombresLibros

Ahora creamos el modelo de similitud donde se aplicará el umbral anteriormente seleccionado.

In [18]:
def crearModeloSimilitud(libros, pel_tfidf,lsi,indice):
    nombreLibros = crearNombresLibros(libros)
    print("Creando enlaces de similitud entre libros")
    print('')
    print('Recomendaciones:')
    print('')
    similar = []
    for i, doc in enumerate(pel_tfidf):
        libroI = nombreLibros[i]
            
        vec_lsi = lsi[doc]
        indice_similitud = indice[vec_lsi]
        similares = []
        for j, elemento in enumerate(libros):
            s = indice_similitud[j]
            libroJ = j
            if (s > UMBRAL_SIMILITUD) and (nombreLibros[i] != nombreLibros[j]):
                similar.append(nombreLibros[j])
                if nombreLibros[i] in similar:
                    pass
                else:
                    print('Libro:','|',libroI.upper(),'|',round(s,3),
                          "similitud ->",'|',nombreLibros[j].upper(),'|')

Por último, ya estamos listos para ver que libros "se parecen" a otros libros.

In [19]:
pel_tfidf   = crearTfIdf(corpus)
(lsi,indice)= crearLSA(corpus,pel_tfidf)
crearModeloSimilitud(libros,pel_tfidf,lsi,indice)

Creación del Modelo Espacio-Vector Tf-Idf
Creación del modelo LSA: Latent Semantic Analysis
Creando enlaces de similitud entre libros

Recomendaciones:

Libro: | EL SEÑOR DE LOS ANILLOS II - LAS DOS TORRES  | 0.976 similitud -> | PATAS ARRIBA  |
Libro: | LOS MITOS DE LA HISTORIA ARGENTINA 2  | 0.866 similitud -> | LOS MITOS DE LA HISTORIA ARGENTINA 3  |
Libro: | LOS MITOS DE LA HISTORIA ARGENTINA 2  | 0.929 similitud -> | LOS MITOS DE LA HISTORIA ARGENTINA 5  |
Libro: | LOS MITOS DE LA HISTORIA ARGENTINA 2  | 0.933 similitud -> | LOS MITOS DE LA HISTORIA ARGENTINA  |
Libro: | EL GRAN DISEÑO  | 0.869 similitud -> | STEPHEN HAWKING: SU VIDA Y OBRA  |
Libro: | HISTORIAS DE DIVÁN  | 0.939 similitud -> | PALABRAS CRUZADAS  |
Libro: | FARENHEIT 451  | 0.991 similitud -> | EL VINO DEL ESTÍO  |
Libro: | EL TRIUNFO DEL SOL  | 0.978 similitud -> | LA RUTA DE LOS VENGADORES  |
Libro: | EL AMOR EN LOS TIEMPOS DEL CÓLERA  | 0.954 similitud -> | EL GENERAL EN SU LABERINTO  |
Libro: | CAVERNAS Y PALA